In [6]:
from scipy.io import wavfile
import wave
import numpy as np
import torchaudio
import os
import pathlib
import torch
from torch.utils.data import TensorDataset, DataLoader
import pyaudio
from torchvision import transforms
from math import ceil

In [3]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE =  32000
CHUNK = 1024
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self,input_data,transform=None):
        self.input_data = input_data
        self.transform=transform

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, index):
        label = self.input_data[index][0]
        audiochunk = self.input_data[index][1]
        if self.transform:
            audiochunk = self.transform(audiochunk)
        return label,audiochunk

In [37]:
def make_chunks(audio_segment, chunk_length):
    """
    Breaks an AudioSegment into chunks that are <chunk_length> milliseconds
    long.
    if chunk_length is 50 then you'll get a list of 50 millisecond long audio
    segments back (except the last one, which can be shorter)
    """
    number_of_chunks = ceil(audio_segment.shape[1] / float(chunk_length))
    for i in range(int(number_of_chunks)):
        yield audio_segment[0,i * chunk_length:(i + 1) * chunk_length]

In [38]:
audio_data = []
i=0

sampleratecheck = 32000
folder = 'Audio' #Assume this is where all audio folders are located. This folder is at same level as script.
abspath = folder
subfolders = os.listdir(abspath)
for audiogroup in subfolders:
    print("Folder:",audiogroup)
    files = os.listdir(os.path.join(abspath,audiogroup))
    print("\tFiles:",files)
    for file in files:
        relpath = os.path.join(os.path.join(abspath,audiogroup),file)
        print(f"\t\tLoading {relpath}")
        data,samplerate = torchaudio.load(relpath)
        #samplerate, data = wavfile.read(relpath)
        if sampleratecheck==None:
            sampleratecheck = samplerate
        else:
            if samplerate != sampleratecheck:
                raise ValueError(f"Sample rate {samplerate} differs from {sampleratecheck}.")
        print(data.shape)
        for chunk in make_chunks(data,CHUNK):
            torchdatachunk = torch.reshape(chunk,(1,-1))
            #np.hstack((chunk,[0]*(CHUNK-len(chunk)))).reshape(1,-1)
            audio_data.append((i,torchdatachunk))
        i+=1

batch_size = 200
train_ds = AudioDataset(audio_data)
train_loader = DataLoader(train_ds, batch_size=batch_size,shuffle=True)

Folder: 0-User
	Files: ['soundsample.wav']
		Loading Audio\0-User\soundsample.wav
torch.Size([1, 144384])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torch.Tensor'>
torch.Size([1024])
<class 'torc

In [25]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(datamean,), std=(datastd,))
])

In [26]:
dataset = AudioDataset(audio_data,transform)


In [27]:
device

device(type='cuda')

In [29]:
i = 0 
for label,data in dataset:
    print(data.device)
    print(data)
    print(data.shape)
    print(np.unique(data))
    print(np.min(data,axis=1))
    print(np.max(data,axis=1))
    i+=1
    if i>1:
        break


cpu
tensor([[[ 1.4526e-03],
         [ 1.4526e-03],
         [-4.5402e-05],
         ...,
         [-4.5402e-05],
         [-3.0415e-03],
         [ 2.9507e-03]]], dtype=torch.float64)
torch.Size([1, 1024, 1])
[-6.03750989e-03 -4.53948299e-03 -3.04145610e-03 -1.54342920e-03
 -4.54023001e-05  1.45262460e-03  2.95065150e-03  4.44867839e-03
  5.94670529e-03]


TypeError: min() received an invalid combination of arguments - got (axis=int, out=NoneType, ), but expected one of:
 * ()
 * (Tensor other)
 * (int dim, bool keepdim)
      didn't match because some of the keywords were incorrect: axis, out
 * (name dim, bool keepdim)
      didn't match because some of the keywords were incorrect: axis, out
